#실험(Experiment) 평가 비교

LangSmith 에서 제공하는 Compare 기능을 활용하여 실험 결과를 쉽게 비교할 수 있습니다.

**참고**

https://docs.smith.langchain.com/cookbook/tracing-examples/traceable#using-the-decorator

In [ ]:
from dotenv import load_dotenv
load_()

In [1]:
from langchain_teddynote import logging
logging.langsmith("CH16-Evaluations")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH16-Evaluations


In [2]:
from myrag import PDFRAG
from langchain_openai import ChatOpenAI

def ask_question_with_llm(llm):
    rag=PDFRAG(
        "data/SPRI_AI_Brief_2023년12월호_F.pdf",
        ChatOpenAI(model="gpt-4o-mini")
    )
    retriever = rag.create_retriever()
    rag_chain = rag.create_chain(retriever)

    def _ask_question(inputs: dict):
        context = retriever.invoke(inputs["question"])
        context = "\n".join([doc.page_content for doc in context])
        return {
            "question": inputs["question"],
            "context": context,
            "answer": rag_chain.invoke(inputs["question"])
        }
    return _ask_question

In [3]:
from langchain_community.chat_models import ChatOllama

# Ollama 모델을 불러옵니다.
ollama = ChatOllama(model="EEVE-Korean-10.8B:latest")

# Ollama 모델 호출
ollama.invoke("안녕하세요?")

AIMessage(content='안녕하세요! 저는 도움이 되고 존중하는 조수로서 여러분의 질문에 최선을 다해 답변하고, 안전하고 유익한 정보를 제공하기 위해 여기에 있습니다.\r\n\r\n질문이나 요청 사항이 있으시면 언제든 물어보세요. 정확하고 유용한 답변을 드리기 위해 최선을 다하겠습니다.\r\n\r\n도움이 되셨길 바라며, 궁금한 점이나 걱정되는 점이 있으면 언제든지 편하게 말씀해 주세요!', response_metadata={'model': 'EEVE-Korean-10.8B:latest', 'created_at': '2024-10-06T14:51:35.3247923Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 10492631300, 'load_duration': 7394053300, 'prompt_eval_count': 50, 'prompt_eval_duration': 124952000, 'eval_count': 84, 'eval_duration': 2967248000}, id='run-4c0b3903-e5d8-408b-8f9d-34acde642586-0')

In [4]:
from langchain_community.chat_models import ChatOllama

# Ollama 모델을 불러옵니다.
ollama = ChatOllama(model="llama3.2:latest")

# Ollama 모델 호출
ollama.invoke("안녕하세요?")

AIMessage(content='하이! 안녕하세요! 어떻게 도와드릴까요?', response_metadata={'model': 'llama3.2:latest', 'created_at': '2024-10-06T14:51:47.8401906Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 3374108800, 'load_duration': 3166296200, 'prompt_eval_count': 28, 'prompt_eval_duration': 50015000, 'eval_count': 14, 'eval_duration': 154335000}, id='run-c80e3ba1-69d0-4c35-a2c8-460d07f18362-0')

In [5]:
from langchain_community.chat_models import ChatOllama
from langchain_openai import ChatOpenAI

gpt_chain = ask_question_with_llm(ChatOpenAI(model="gpt-4o-mini", temperature=0))
ollama_chain = ask_question_with_llm(ChatOllama(model="EEVE-Korean-10.8B:latest"))

In [8]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator

# qa 평가자 생성
cot_qa_evalulator = LangChainStringEvaluator(
    "cot_qa",
    config={"llm": ChatOpenAI(model="gpt-4o-mini", temperature=0)},
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],
        "reference": run.outputs["context"],
        "input": example.inputs["question"],
    },
)


In [9]:
dataset_name = "RAG_EVAL_DATASET_teddynote"
# 평가 실행
experiment_results1 = evaluate(
    gpt_chain,
    data=dataset_name,
    evaluators=[cot_qa_evalulator],
    experiment_prefix="MODEL_COMPARE_EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "GPT-4o-mini 평가 (cot_qa)",
    },
)

View the evaluation results for experiment: 'MODEL_COMPARE_EVAL-b7c9d702' at:
https://smith.langchain.com/o/2d0ce887-3f7f-59af-8d5e-12c1371ef5d5/datasets/334d943e-3086-4d03-91ae-4b5145224ffc/compare?selectedSessions=43196ad0-54f8-43a0-aa03-e01c4b8e865c




0it [00:00, ?it/s]

Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running evaluator <DynamicRunEvaluator evaluate> on run 817f431b-1ed4-49ee-9e24-02c022e01d37: KeyError('answer')
Traceback (most recent call last):
  File "c:\Users\skyop\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-gLkynrUQ-py3.11\Lib\site-packages\langsmith\evaluation\_runner.py", line 1345, in _run_evaluators
    evaluator_response = evaluator.evaluate_run(
                         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\skyop\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-gLkynrUQ-py3.11\Lib\site-packages\langsmith\evaluation\evaluator.py", line 327, in evaluate_run
    result = self.func(
             ^^^^^^^^^^
  File "c:\Users\skyop\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-gLkynrUQ-py3.11\Lib\site-packages\langsmith\run_helpers.py", line 646, in wrapper
    raise e
  File "c:\Us

In [10]:
# 평가 실행
experiment_results2 = evaluate(
    ollama_chain,
    data=dataset_name,
    evaluators=[cot_qa_evalulator],
    experiment_prefix="MODEL_COMPARE_EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "Ollama(EEVE-Korean-10.8B:latest) 평가 (cot_qa)",
    },
)

View the evaluation results for experiment: 'MODEL_COMPARE_EVAL-2e17983d' at:
https://smith.langchain.com/o/2d0ce887-3f7f-59af-8d5e-12c1371ef5d5/datasets/334d943e-3086-4d03-91ae-4b5145224ffc/compare?selectedSessions=c9c22a10-e18c-45d5-91cd-e9edd1154040




0it [00:00, ?it/s]

Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running evaluator <DynamicRunEvaluator evaluate> on run c2451f11-f046-4f85-ba30-ea596d3a0227: KeyError('answer')
Traceback (most recent call last):
  File "c:\Users\skyop\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-gLkynrUQ-py3.11\Lib\site-packages\langsmith\evaluation\_runner.py", line 1345, in _run_evaluators
    evaluator_response = evaluator.evaluate_run(
                         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\skyop\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-gLkynrUQ-py3.11\Lib\site-packages\langsmith\evaluation\evaluator.py", line 327, in evaluate_run
    result = self.func(
             ^^^^^^^^^^
  File "c:\Users\skyop\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-gLkynrUQ-py3.11\Lib\site-packages\langsmith\run_helpers.py", line 646, in wrapper
    raise e
  File "c:\Users\skyop\AppData\Local\pypoetry\Cache\vir